<a href="https://colab.research.google.com/github/MoamenTlili/nlp-AI/blob/main/P3_ENSEMBLE_MODEL_Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 9.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from sklearn.metrics import f1_score, classification_report, accuracy_score

In [ ]:
df = pd.read_csv('/content/sample_data/MLMA_hate_speech.csv')

# Split the data
train_df, test_val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])
val_df, test_df = train_test_split(test_val_df, test_size=0.5, random_state=42, stratify=test_val_df['target'])

# Prepare test data
test_texts = test_df['tweet'].values
test_labels = pd.get_dummies(test_df['target']).values

# Function to preprocess the text data
def preprocess_data(texts, tokenizer, max_length=128):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='tf'
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
    return tf.concat(input_ids, axis=0), tf.concat(attention_masks, axis=0)

labse_model_path = '/content/drive/MyDrive/fine_tuned_mult_models_tf/p3_TASK3_labert_Mult_full'
xlm_roberta_model_path = '/content/drive/MyDrive/fine_tuned_mult_models_tf/p3_TASK3_xlmro_Mult_full'
mbert_model_path = '/content/drive/MyDrive/fine_tuned_mult_models_tf/p3_TASK3_mbert_Mult_full'
xlm_model_path = '/content/drive/MyDrive/fine_tuned_mult_models_tf/p3_TASK3_xlm_Mult_full'

labse_model = tf.saved_model.load(labse_model_path)
xlm_roberta_model = tf.saved_model.load(xlm_roberta_model_path)
mbert_model = tf.saved_model.load(mbert_model_path)
xlm_model = tf.saved_model.load(xlm_model_path)

serving_fn_labse = labse_model.signatures['serving_default']
serving_fn_xlm_roberta = xlm_roberta_model.signatures['serving_default']
serving_fn_mbert = mbert_model.signatures['serving_default']
serving_fn_xlm = xlm_model.signatures['serving_default']

labse_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")
xlm_roberta_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
mbert_tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
xlm_tokenizer = AutoTokenizer.from_pretrained('xlm-mlm-enfr-1024')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

In [ ]:
labse_input_ids, labse_attention_masks = preprocess_data(test_texts, labse_tokenizer)
xlm_roberta_input_ids, xlm_roberta_attention_masks = preprocess_data(test_texts, xlm_roberta_tokenizer)
mbert_input_ids, mbert_attention_masks = preprocess_data(test_texts, mbert_tokenizer)
xlm_input_ids, xlm_attention_masks = preprocess_data(test_texts, xlm_tokenizer)

labse_inputs = {
    'input_word_ids': labse_input_ids,
    'input_mask': labse_attention_masks
}


xlm_roberta_inputs = {
    'input_ids': xlm_roberta_input_ids,
    'attention_mask': xlm_roberta_attention_masks
}

mbert_inputs = {
    'input_ids': mbert_input_ids,
    'attention_mask': mbert_attention_masks
}

xlm_inputs = {
    'input_ids': xlm_input_ids,
    'attention_mask': xlm_attention_masks
}

predictions_labse = serving_fn_labse(
    input_word_ids=labse_input_ids,
    input_mask=labse_attention_masks
)['dense_3']

predictions_xlm_roberta = serving_fn_xlm_roberta(
    input_word_ids=xlm_roberta_input_ids,
    input_mask=xlm_roberta_attention_masks
)['dense_3']
predictions_mbert = serving_fn_mbert(
    input_ids=mbert_input_ids,
    attention_mask=mbert_attention_masks
)['dense_1']

predictions_xlm = serving_fn_xlm(
    input_word_ids=xlm_input_ids,
    input_mask=xlm_attention_masks
)['dense_3']

In [ ]:
def evaluate_multiclass_predictions(predictions, true_labels):
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(true_labels, axis=1)

    macro_f1 = f1_score(true_classes, predicted_classes, average='macro')
    micro_f1 = f1_score(true_classes, predicted_classes, average='micro')

    report = classification_report(true_classes, predicted_classes, target_names=test_df['target'].unique())

    return macro_f1, micro_f1, report

# Evaluate each model
macro_f1_labse, micro_f1_labse, report_labse = evaluate_multiclass_predictions(predictions_labse, test_labels)
macro_f1_xlm_roberta, micro_f1_xlm_roberta, report_xlm_roberta = evaluate_multiclass_predictions(predictions_xlm_roberta, test_labels)
macro_f1_mbert, micro_f1_mbert, report_mbert = evaluate_multiclass_predictions(predictions_mbert, test_labels)
macro_f1_xlm, micro_f1_xlm, report_xlm = evaluate_multiclass_predictions(predictions_xlm, test_labels)

# Ensemble Methods
# Simple Averaging
avg_predictions = (predictions_labse + predictions_xlm_roberta + predictions_mbert + predictions_xlm) / 4

# Weighted Averaging => try this next [0.267, 0.252, 0.252, 0.229] !!!
weights = [0.33, 0.33, 0.34, 0.30]
weighted_avg_predictions = (weights[0] * predictions_labse +
                            weights[1] * predictions_xlm_roberta +
                            weights[2] * predictions_mbert +
                            weights[3] * predictions_xlm)

macro_f1_avg, micro_f1_avg, report_avg = evaluate_multiclass_predictions(avg_predictions, test_labels)
macro_f1_weighted, micro_f1_weighted, report_weighted = evaluate_multiclass_predictions(weighted_avg_predictions, test_labels)

print(f"Macro F1 Score (Averaging): {macro_f1_avg}, Micro F1 Score (Averaging): {micro_f1_avg}")
print(f"Classification Report (Averaging):\n{report_avg}\n")

print(f"Macro F1 Score (Weighted Averaging): {macro_f1_weighted}, Micro F1 Score (Weighted Averaging): {micro_f1_weighted}")
print(f"Classification Report (Weighted Averaging):\n{report_weighted}\n")

# Majority Voting
# Convert model predictions to class indices
predicted_classes_labse = np.argmax(predictions_labse, axis=1)
predicted_classes_xlm_roberta = np.argmax(predictions_xlm_roberta, axis=1)
predicted_classes_mbert = np.argmax(predictions_mbert, axis=1)
predicted_classes_xlm = np.argmax(predictions_xlm, axis=1)

majVot_predictions = np.array([
    np.bincount([pred_labse, pred_xlm_roberta, pred_mbert, pred_xlm]).argmax()
    for pred_labse, pred_xlm_roberta, pred_mbert, pred_xlm in zip(
        predicted_classes_labse, predicted_classes_xlm_roberta, predicted_classes_mbert, predicted_classes_xlm)
])

true_classes = np.argmax(test_labels, axis=1)
macro_f1_majority = f1_score(true_classes, majVot_predictions, average='macro')
micro_f1_majority = f1_score(true_classes, majVot_predictions, average='micro')
report_majority = classification_report(true_classes, majVot_predictions, target_names=test_df['target'].unique())

print(f"Macro F1 Score (Majority Voting): {macro_f1_majority}, Micro F1 Score (Majority Voting): {micro_f1_majority}")
print(f"Classification Report (Majority Voting):\n{report_majority}\n")

Macro F1 Score (Averaging): 0.7107739647891469, Micro F1 Score (Averaging): 0.7337975361542581
Classification Report (Averaging):
                    precision    recall  f1-score   support

sexual_orientation       0.71      0.89      0.79       236
             other       0.63      0.84      0.72       185
            origin       0.85      0.76      0.80       804
        disability       0.67      0.52      0.59       495
          religion       0.39      0.79      0.52        43
            gender       0.75      0.96      0.84       104

          accuracy                           0.73      1867
         macro avg       0.67      0.79      0.71      1867
      weighted avg       0.75      0.73      0.73      1867


Macro F1 Score (Weighted Averaging): 0.7104773702265083, Micro F1 Score (Weighted Averaging): 0.7332619175147295
Classification Report (Weighted Averaging):
                    precision    recall  f1-score   support

sexual_orientation       0.71      0.90      0.7

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
meta_features = np.stack([predictions_labse, predictions_xlm_roberta, predictions_mbert, predictions_xlm], axis=2)

meta_features = meta_features.reshape(meta_features.shape[0], -1)

lr_meta_learner = LogisticRegression(max_iter=1000, multi_class='ovr')
lr_meta_learner.fit(meta_features, np.argmax(test_labels, axis=1))

meta_predictions_lr = lr_meta_learner.predict(meta_features)

macro_f1_lr = f1_score(np.argmax(test_labels, axis=1), meta_predictions_lr, average='macro')
micro_f1_lr = f1_score(np.argmax(test_labels, axis=1), meta_predictions_lr, average='micro')
report_lr = classification_report(np.argmax(test_labels, axis=1), meta_predictions_lr, target_names=test_df['target'].unique())

print(f"Logistic Regression Meta-Learner - Macro F1 Score: {macro_f1_lr}, Micro F1 Score: {micro_f1_lr}")
print(f"Classification Report (Logistic Regression):\n{report_lr}")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic Regression Meta-Learner - Macro F1 Score: 0.7147419516837421, Micro F1 Score: 0.7471880021424746
Classification Report (Logistic Regression):
                    precision    recall  f1-score   support

sexual_orientation       0.76      0.87      0.81       236
             other       0.63      0.75      0.68       185
            origin       0.83      0.81      0.82       804
        disability       0.66      0.56      0.61       495
          religion       0.52      0.53      0.53        43
            gender       0.75      0.94      0.83       104

          accuracy                           0.75      1867
         macro avg       0.69      0.74      0.71      1867
      weighted avg       0.75      0.75      0.74      1867



In [ ]:
from sklearn.svm import SVC

svm_meta_learner = SVC(kernel='linear', decision_function_shape='ovr')
svm_meta_learner.fit(meta_features, np.argmax(test_labels, axis=1))

meta_predictions_svm = svm_meta_learner.predict(meta_features)

macro_f1_svm = f1_score(np.argmax(test_labels, axis=1), meta_predictions_svm, average='macro')
micro_f1_svm = f1_score(np.argmax(test_labels, axis=1), meta_predictions_svm, average='micro')
report_svm = classification_report(np.argmax(test_labels, axis=1), meta_predictions_svm, target_names=test_df['target'].unique())

print(f"SVM Meta-Learner - Macro F1 Score: {macro_f1_svm}, Micro F1 Score: {micro_f1_svm}")
print(f"Classification Report (SVM):\n{report_svm}")

SVM Meta-Learner - Macro F1 Score: 0.7199133199812978, Micro F1 Score: 0.7487948580610605
Classification Report (SVM):
                    precision    recall  f1-score   support

sexual_orientation       0.78      0.82      0.80       236
             other       0.63      0.82      0.71       185
            origin       0.82      0.81      0.82       804
        disability       0.68      0.56      0.61       495
          religion       0.51      0.58      0.54        43
            gender       0.75      0.93      0.83       104

          accuracy                           0.75      1867
         macro avg       0.70      0.75      0.72      1867
      weighted avg       0.75      0.75      0.75      1867



!!!

!!!!
!!

!!!!

!!!!!

!!!!!


!!!!!

In [ ]:
def evaluate_multiclass_predictions(predictions, true_labels):
    if len(true_labels.shape) > 1:
        true_classes = np.argmax(true_labels, axis=1)
    else:
        true_classes = true_labels

    predicted_classes = np.argmax(predictions, axis=1)

    macro_f1 = f1_score(true_classes, predicted_classes, average='macro')
    micro_f1 = f1_score(true_classes, predicted_classes, average='micro')

    report = classification_report(true_classes, predicted_classes, target_names=test_df['target'].unique())

    return macro_f1, micro_f1, report


# Evaluate each model
macro_f1_labse, micro_f1_labse, report_labse = evaluate_multiclass_predictions(predictions_labse, test_labels)
macro_f1_xlm_roberta, micro_f1_xlm_roberta, report_xlm_roberta = evaluate_multiclass_predictions(predictions_xlm_roberta, test_labels)
macro_f1_mbert, micro_f1_mbert, report_mbert = evaluate_multiclass_predictions(predictions_mbert, test_labels)
macro_f1_xlm, micro_f1_xlm, report_xlm = evaluate_multiclass_predictions(predictions_xlm, test_labels)

# Ensemble Methods
# Simple Averaging
avg_predictions = (predictions_labse + predictions_xlm_roberta + predictions_mbert + predictions_xlm) / 4

# Weighted Averaging
weights = [0.33, 0.33, 0.34, 0.30]  # Adjust weights if needed
weighted_avg_predictions = (weights[0] * predictions_labse +
                            weights[1] * predictions_xlm_roberta +
                            weights[2] * predictions_mbert +
                            weights[3] * predictions_xlm)

# Evaluate Ensembles
macro_f1_avg, micro_f1_avg, report_avg = evaluate_multiclass_predictions(avg_predictions, test_labels)
macro_f1_weighted, micro_f1_weighted, report_weighted = evaluate_multiclass_predictions(weighted_avg_predictions, test_labels)

# Print Ensemble Results
print(f"Macro F1 Score (Averaging): {macro_f1_avg}, Micro F1 Score (Averaging): {micro_f1_avg}")
print(f"Classification Report (Averaging):\n{report_avg}\n")

print(f"Macro F1 Score (Weighted Averaging): {macro_f1_weighted}, Micro F1 Score (Weighted Averaging): {micro_f1_weighted}")
print(f"Classification Report (Weighted Averaging):\n{report_weighted}\n")

# Majority Voting
# Convert model predictions to class indices
predicted_classes_labse = np.argmax(predictions_labse, axis=1)
predicted_classes_xlm_roberta = np.argmax(predictions_xlm_roberta, axis=1)
predicted_classes_mbert = np.argmax(predictions_mbert, axis=1)
predicted_classes_xlm = np.argmax(predictions_xlm, axis=1)

# Apply majority voting
majVot_predictions = np.array([
    np.bincount([pred_labse, pred_xlm_roberta, pred_mbert, pred_xlm]).argmax()
    for pred_labse, pred_xlm_roberta, pred_mbert, pred_xlm in zip(
        predicted_classes_labse, predicted_classes_xlm_roberta, predicted_classes_mbert, predicted_classes_xlm)
])

# Evaluate Majority Voting
true_classes = np.argmax(test_labels, axis=1)
macro_f1_majority = f1_score(true_classes, majVot_predictions, average='macro')
micro_f1_majority = f1_score(true_classes, majVot_predictions, average='micro')
report_majority = classification_report(true_classes, majVot_predictions, target_names=test_df['target'].unique())

# Print Majority Voting Results
print(f"Macro F1 Score (Majority Voting): {macro_f1_majority}, Micro F1 Score (Majority Voting): {micro_f1_majority}")
print(f"Classification Report (Majority Voting):\n{report_majority}\n")

Macro F1 Score (Averaging): 0.7107739647891469, Micro F1 Score (Averaging): 0.7337975361542581
Classification Report (Averaging):
                    precision    recall  f1-score   support

sexual_orientation       0.71      0.89      0.79       236
             other       0.63      0.84      0.72       185
            origin       0.85      0.76      0.80       804
        disability       0.67      0.52      0.59       495
          religion       0.39      0.79      0.52        43
            gender       0.75      0.96      0.84       104

          accuracy                           0.73      1867
         macro avg       0.67      0.79      0.71      1867
      weighted avg       0.75      0.73      0.73      1867


Macro F1 Score (Weighted Averaging): 0.7104773702265083, Micro F1 Score (Weighted Averaging): 0.7332619175147295
Classification Report (Weighted Averaging):
                    precision    recall  f1-score   support

sexual_orientation       0.71      0.90      0.7

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

# Prepare the meta-learner features (stacked predictions from all models)
meta_features = np.stack([predictions_labse, predictions_xlm_roberta, predictions_mbert, predictions_xlm], axis=2)

# Convert the stacked predictions into the shape [n_samples, n_models]
meta_features = meta_features.reshape(meta_features.shape[0], -1)

# Train a logistic regression as the meta-learner
lr_meta_learner = LogisticRegression(max_iter=1000, multi_class='ovr')
lr_meta_learner.fit(meta_features, np.argmax(test_labels, axis=1))

# Make predictions using the meta-learner
meta_predictions_lr = lr_meta_learner.predict(meta_features)

# Evaluate
macro_f1_lr = f1_score(np.argmax(test_labels, axis=1), meta_predictions_lr, average='macro')
micro_f1_lr = f1_score(np.argmax(test_labels, axis=1), meta_predictions_lr, average='micro')
report_lr = classification_report(np.argmax(test_labels, axis=1), meta_predictions_lr, target_names=test_df['target'].unique())

# Print Results
print(f"Logistic Regression Meta-Learner - Macro F1 Score: {macro_f1_lr}, Micro F1 Score: {micro_f1_lr}")
print(f"Classification Report (Logistic Regression):\n{report_lr}")

Logistic Regression Meta-Learner - Macro F1 Score: 0.7147419516837421, Micro F1 Score: 0.7471880021424746
Classification Report (Logistic Regression):
                    precision    recall  f1-score   support

sexual_orientation       0.76      0.87      0.81       236
             other       0.63      0.75      0.68       185
            origin       0.83      0.81      0.82       804
        disability       0.66      0.56      0.61       495
          religion       0.52      0.53      0.53        43
            gender       0.75      0.94      0.83       104

          accuracy                           0.75      1867
         macro avg       0.69      0.74      0.71      1867
      weighted avg       0.75      0.75      0.74      1867



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
from sklearn.svm import SVC

# Train a Support Vector Machine as the meta-learner
svm_meta_learner = SVC(kernel='linear', decision_function_shape='ovr')
svm_meta_learner.fit(meta_features, np.argmax(test_labels, axis=1))

# Make predictions using the SVM meta-learner
meta_predictions_svm = svm_meta_learner.predict(meta_features)

# Evaluate
macro_f1_svm = f1_score(np.argmax(test_labels, axis=1), meta_predictions_svm, average='macro')
micro_f1_svm = f1_score(np.argmax(test_labels, axis=1), meta_predictions_svm, average='micro')
report_svm = classification_report(np.argmax(test_labels, axis=1), meta_predictions_svm, target_names=test_df['target'].unique())

# Print Results
print(f"SVM Meta-Learner - Macro F1 Score: {macro_f1_svm}, Micro F1 Score: {micro_f1_svm}")
print(f"Classification Report (SVM):\n{report_svm}")

SVM Meta-Learner - Macro F1 Score: 0.7199133199812978, Micro F1 Score: 0.7487948580610605
Classification Report (SVM):
                    precision    recall  f1-score   support

sexual_orientation       0.78      0.82      0.80       236
             other       0.63      0.82      0.71       185
            origin       0.82      0.81      0.82       804
        disability       0.68      0.56      0.61       495
          religion       0.51      0.58      0.54        43
            gender       0.75      0.93      0.83       104

          accuracy                           0.75      1867
         macro avg       0.70      0.75      0.72      1867
      weighted avg       0.75      0.75      0.75      1867



In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train a Random Forest as the meta-learner
rf_meta_learner = RandomForestClassifier(n_estimators=100, random_state=42)
rf_meta_learner.fit(meta_features, np.argmax(test_labels, axis=1))

# Make predictions using the Random Forest meta-learner
meta_predictions_rf = rf_meta_learner.predict(meta_features)

# Evaluate
macro_f1_rf = f1_score(np.argmax(test_labels, axis=1), meta_predictions_rf, average='macro')
micro_f1_rf = f1_score(np.argmax(test_labels, axis=1), meta_predictions_rf, average='micro')
report_rf = classification_report(np.argmax(test_labels, axis=1), meta_predictions_rf, target_names=test_df['target'].unique())

# Print Results
print(f"Random Forest Meta-Learner - Macro F1 Score: {macro_f1_rf}, Micro F1 Score: {micro_f1_rf}")
print(f"Classification Report (Random Forest):\n{report_rf}")

Random Forest Meta-Learner - Macro F1 Score: 1.0, Micro F1 Score: 1.0
Classification Report (Random Forest):
                    precision    recall  f1-score   support

sexual_orientation       1.00      1.00      1.00       236
             other       1.00      1.00      1.00       185
            origin       1.00      1.00      1.00       804
        disability       1.00      1.00      1.00       495
          religion       1.00      1.00      1.00        43
            gender       1.00      1.00      1.00       104

          accuracy                           1.00      1867
         macro avg       1.00      1.00      1.00      1867
      weighted avg       1.00      1.00      1.00      1867

